# Collaborative filtering practice

In this homework you will test different collaborative filtering (CF) approaches on famous Movielens dataset.

In class we implemented item2item CF, so this time let's use **user2user** approach.

## Task 0: Dataset (5 points)

Load [movielens](https://grouplens.org/datasets/movielens/) dataset using [scikit surprise](https://surprise.readthedocs.io/en/stable/dataset.html)

Split dataset to train and validation parts.

Don't forget to encode users and items from 0 to maximum!

In [10]:
!pip install scikit-surprise

In [11]:
import pandas as pd
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
import numpy as np

In [12]:
data = Dataset.load_builtin('ml-100k')

df = pd.DataFrame(data.raw_ratings, columns=['user_id', 'item_id', 'rating', 'timestamp'])

# Энкодим пользователей и айтемы от 0 до max
df['user_id'] = df['user_id'].astype('category').cat.codes
df['item_id'] = df['item_id'].astype('category').cat.codes

# Объект класса Reader, необходимый для определения шкалы рейтинга
reader = Reader(rating_scale=(1, 5))

# Загружаем датасет в формат surprice, применяя reader с заданной шкалой рейтинга
surprise_data = Dataset.load_from_df(df[['user_id', 'item_id', 'rating']], reader)

# Разбиваем датасет на тренировочную и тестовую выборки
train, test = train_test_split(surprise_data, test_size=0.2, random_state=42)
df.head(50)

,user_id,item_id,rating,timestamp
0,107,842,3.0,881250949
1,96,909,3.0,891717742
2,134,991,1.0,878887116
3,161,1139,2.0,880606923
4,74,957,1.0,886397596
5,220,1099,4.0,884182806
6,18,867,2.0,881171488
7,171,1089,5.0,891628467
8,229,1074,3.0,886324817
9,555,1527,3.0,883603013


## Task 1: Similarities (5 points each)

You need to implement 3 similarity functions:
1. Dot product (intersection)
1. Jaccard index (intersection over union)
1. Pearson correlation
1. Pearson correlation with decreasing coefficient

In [13]:
def sim_dot(left, right) -> float:
    '''Dot product similarity

    Args:
        left: first user ratings
        right: second user ratings

    Retruns:
        Similarity score for this pair
    '''
    mask = (left != 0) & (right != 0)

    # Если нет пересечения, возвращаем 0
    if not np.any(mask):
        return 0.0

    # Используем маску для получения элементов и вычисления скалярного произведения
    sim = np.dot(left[mask], right[mask])
    return sim

In [14]:
def sim_jacc(left, right) -> float:
    '''Jaccard index similarity

    Args:
        left: first user ratings
        right: second user ratings

    Retruns:
        Similarity score for this pair
    '''
    # Преобразуем в булевы массивы, где True означает, что оценка не равна 0
    left_nonzero = left != 0
    right_nonzero = right != 0

    # Находим пересечение и объединение
    intersection = np.sum(left_nonzero & right_nonzero)
    union = np.sum(left_nonzero | right_nonzero)

    # Если объединение равно 0, возвращаем 0.0
    if union == 0:
        return 0.0

    # Рассчитываем коэффициент Жаккара
    sim = intersection / union

    return sim

In [15]:
def sim_pearson(left, right) -> float:
    '''Pearson correlation similarity

    Args:
        left: first user ratings
        right: second user ratings

    Retruns:
        Similarity score for this pair
    '''

    # Создаем маску для ненулевых оценок
    mask = (left != 0) & (right != 0)

    # Извлекаем оценки, соответствующие маске
    scores1 = left[mask]
    scores2 = right[mask]

    # Проверяем, достаточно ли оценок для расчета
    if len(scores1) < 2 or len(scores2) < 2:
        return 0.0

    # Проверяем, не равна ли первая оценка среднему значению
    if scores1[0] == scores1.mean() or scores2[0] == scores2.mean():
        return 0.0

    # Вычисляем коэффициент корреляции Пирсона
    if len(scores1) == 0 or len(scores2) == 0:
        return 0.0

    sim = np.corrcoef(scores1, scores2)[0, 1]

    # Возвращаем 0.0, если значение NaN
    if np.isnan(sim):
        return 0.0

    return sim

In [16]:
def sim_pearson_decreasing(left, right) -> float:
    '''Pearson correlation similarity which decreases on small intersection

    Args:
        left: first user ratings
        right: second user ratings

    Retruns:
        Similarity score for this pair
    '''

    # Создаем маску для ненулевых оценок
    mask = (left != 0) & (right != 0)

    # Извлекаем оценки, соответствующие маске
    scores1 = left[mask]
    scores2 = right[mask]

    # Проверяем, достаточно ли оценок для расчета
    if len(scores1) < 2 or len(scores2) < 2:
        return 0.0

    # Проверяем, не равна ли первая оценка среднему значению
    if scores1[0] == scores1.mean() or scores2[0] == scores2.mean():
        return 0.0

    # Вычисляем коэффициент корреляции Пирсона
    if len(scores1) == 0 or len(scores2) == 0:
        return 0.0

    r = np.corrcoef(scores1, scores2)[0, 1]

    # Возвращаем 0.0, если значение NaN
    if np.isnan(r):
        return 0.0

    # Уменьшаем схожесть в случае, если пересечение по элементам слишком малое
    intersection_count = np.sum(mask)
    sim = min(intersection_count / 50, 1) * r

    return sim

## Task 2: Collaborative filtering algorithm (5 points each)

Now you have several options to use similarities for ratings prediction:
1. Simple averaging
1. Mean corrected averaging

In [17]:
class UserBasedCf:
  '''User2user collaborative filtering algorithm'''
  def __init__(self, sim_fn, mean_correct: bool = False):
        self.sim_fn = sim_fn
        self.mean_correct = mean_correct

  def calc_sim_matrix(self, feedbacks):
        '''Fills matrix of user similarities

        Args:
            feedbacks: numpy array with ratings
        '''
        self.feedbacks = feedbacks

        users_count = feedbacks.shape[0]
        sim_m = np.zeros((users_count, users_count))

        # Заполняем матрицу похожестей, используя входную функцию
        for i, left in enumerate(feedbacks):
          for j, right in enumerate(feedbacks):
            sim_m[i,j] = self.sim_fn(left, right)

        # Округляем до 3 знаков после запятой
        self.sim_matrix = sim_m


  def recommend(self, user: int, n: int):
      '''Computes most relevant unseen items for the user

      Args:
          user: user_id for which to provide recommendations
          n: how many items to return
      '''
      # Получаем похожести для заданного пользователя
      sim = self.sim_matrix[user]

      # Берем топ k соседей
      k = 10
      top_k_neigh = np.argsort(sim)[::-1][:k]

      # Массивы для хранения числителя и знаменателя
      numerator = np.zeros(self.feedbacks.shape[1])
      denominator = np.zeros(self.feedbacks.shape[1])

      for neigh in top_k_neigh:
          if neigh == user:
              continue

          # Маска неоцененных айтемов текущего пользователя
          unseen_mask = self.feedbacks[user] == 0

          if not self.mean_correct:
              # Simple averaging
              numerator[unseen_mask] += sim[neigh] * self.feedbacks[neigh][unseen_mask]
              denominator[unseen_mask] += np.abs(sim[neigh])

          else:
              # Mean corrected averaging
              # Считаем среднюю оценку соседа
              neigh_scores = self.feedbacks[neigh]
              neigh_mask = neigh_scores != 0
              neigh_mean = neigh_scores[neigh_mask].mean()

              # Считаем числитель и знаменатель
              numerator[unseen_mask] += sim[neigh] * (self.feedbacks[neigh][unseen_mask] - neigh_mean)
              denominator[unseen_mask] += np.abs(sim[neigh])

      # Формируем предсказания
      predictions = np.zeros(self.feedbacks.shape[1])
      valid_mask = denominator != 0

      if not self.mean_correct:
          predictions[valid_mask] = numerator[valid_mask] / denominator[valid_mask]
      else:
          # Считаем среднюю оценку текущего пользователя
          user_scores = self.feedbacks[user]
          user_mask = user_scores != 0
          user_mean = user_scores[user_mask].mean()

          predictions[valid_mask] = user_mean + numerator[valid_mask] / denominator[valid_mask]

      # Получаем индексы неоцененных айтемов
      unseen_indices = np.where(self.feedbacks[user] == 0)[0]

      # Формируем рекомендации
      recommendations = [(predictions[i], i) for i in unseen_indices]
      recommendations.sort(reverse=True)

      return recommendations[:n]



This way you have got 6 different recommendation methods (each of two CF modes can be used with 3 similarity scores).

## Task 3: Apply models

1. For all 6 possible algorithm variations train it and compute recomendations for validation part. (10 points)

In [18]:
feedbacks = np.zeros((train.n_users, train.n_items))
for uid, iid, rating in train.all_ratings():
    feedbacks[uid][iid] = rating

# List of similarity functions
similarity_functions = [sim_dot, sim_jacc, sim_pearson, sim_pearson_decreasing]
mean_corrections = [True, False]

# Train and test all combinations
for sim_fn in similarity_functions:
    for mean_correct in mean_corrections:
        # Initialize the UserBasedCf class
        user_cf = UserBasedCf(sim_fn=sim_fn, mean_correct=mean_correct)
        # Calculate the similarity matrix
        user_cf.calc_sim_matrix(feedbacks)

        # Generate recommendations for users in the validation set
        print(f"Recommendations using {sim_fn.__name__} with mean correction = {mean_correct}")
        for user in range(train.n_users):
            recommendations = user_cf.recommend(user, n=5)
            print(f"User {user}: Recommended items: {recommendations}")
        print("\n" + "=" * 50 + "\n")

Streaming output truncated to the last 5000 lines.
User 681: Recommended items: [(1.953905271425705, 230), (1.94714964741433, 164), (1.904330494444129, 559), (1.8479668323791278, 216), (1.8303116245690987, 528)]
User 682: Recommended items: [(2.0995808009378427, 305), (2.0962486417028794, 6), (1.879467318855229, 83), (1.784366123822746, 154), (1.729538298900592, 436)]
User 683: Recommended items: [(2.3071853711721664, 129), (2.145656401132819, 88), (2.0674643434973468, 76), (2.000088378414349, 183), (1.9288390086561547, 41)]
User 684: Recommended items: [(3.5858268285620127, 6), (2.4347421622803718, 76), (2.3891908518992944, 204), (2.229789546036213, 285), (2.1557046129717747, 295)]
User 685: Recommended items: [(2.5817583105100543, 8), (2.34822703644736, 305), (2.2102654975856897, 166), (2.1825741955502815, 92), (1.8100028430585526, 122)]
User 686: Recommended items: [(3.5566696600012975, 41), (3.1012919777916164, 267), (2.708208135073507, 88), (2.6188478279050327, 305), (2.5025767855

2. Which metrics do you want to use? Why? (5 points)

NDCG, MRR, MAP@k

3. Show that your implementation is relevant by computing metrics. Compare algorithms. (15 points)

# Task 4: Your favorite films

1. Choose from 10 to 50 films rated by you (you can export it from IMDB or kinopoisk) which are presented in Movielens dataset. </br> Print them in human readable form (5 points)

In [19]:
# your code here

2. Compute top 10 recomendations based on this films for each of 6 methods implemented. Print them in **human readable from** (5 points)

In [20]:
# your code here

3. Rate films that was recommended in previous step (by title, description, trailer). For each algorithm compute metrics based on ratings you put.

_Your ratings_

# Task 5: Conclusion (10 points)

Compare all methods based on both dataset (metrics) and your personal recomendations.

Which algorithm is the best? Why?

Was recommedations different? Which set of recomendations you like the most?

What differences in algorithms have you noted?

_Your conclusion_